In [ ]:
# [ベースライン] NFL Draft Prediction - ローカル環境版

本ノートブックでは、ベースラインモデルをローカル環境で実行し、**ベースラインスコア0.80792を超える**ことを目標とします。

## 目標
- ベースラインモデルの理解と実行
- 提出ファイルの作成
- スコア0.80792の達成確認
- 改善ポイントの特定


In [ ]:
## 1. セットアップ


In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

# 設定
warnings.filterwarnings('ignore')
np.random.seed(42)

print("ライブラリの読み込み完了")


In [ ]:
## 2. データ読み込み


In [ ]:
# データ読み込み
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission_6_20修正.csv')

print('Train:', train.shape)
print('Test:', test.shape)
print('Sample submission:', sample_submission.shape)

# データの確認
print("\n=== 目的変数の確認 ===")
print(f"Drafted平均値: {train['Drafted'].mean():.5f}")
print(f"Drafted分布:")
print(train['Drafted'].value_counts().sort_index())


In [ ]:
## 3. 欠損値の確認


In [ ]:
# 欠損値の確認
print("=== 訓練データの欠損値 ===")
train_missing = train.isnull().sum()
print(train_missing[train_missing > 0])

print("\n=== テストデータの欠損値 ===")
test_missing = test.isnull().sum()
print(test_missing[test_missing > 0])

# 欠損率の計算
missing_ratio_train = train_missing[train_missing > 0] / len(train) * 100
missing_ratio_test = test_missing[test_missing > 0] / len(test) * 100

print(f"\n=== 欠損率(%) ===")
print("Train:", missing_ratio_train.round(1))
print("Test:", missing_ratio_test.round(1))


In [ ]:
## 4. 前処理


In [ ]:
# 前処理関数の定義
def preprocess_data(train_df, test_df):
    # データのコピーを作成
    train_processed = train_df.copy()
    test_processed = test_df.copy()
    
    # 全データを結合して一括前処理
    all_data = pd.concat([train_processed, test_processed], ignore_index=True)
    
    # 1. 欠損値処理（中央値で補完）
    numerical_cols = all_data.select_dtypes(include=[np.number]).columns.tolist()
    numerical_cols = [col for col in numerical_cols if col not in ['Id', 'Drafted']]
    
    for col in numerical_cols:
        median_val = all_data[col].median()
        all_data[col].fillna(median_val, inplace=True)
    
    # 2. カテゴリ変数のラベルエンコーディング
    categorical_cols = all_data.select_dtypes(include=['object']).columns.tolist()
    
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        all_data[col] = le.fit_transform(all_data[col].astype(str))
        label_encoders[col] = le
    
    # 3. データを分割
    train_len = len(train_processed)
    train_processed = all_data[:train_len].copy()
    test_processed = all_data[train_len:].copy()
    
    # 4. 特徴量とターゲットを分離
    feature_cols = [col for col in train_processed.columns if col not in ['Id', 'Drafted']]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed['Drafted']
    X_test = test_processed[feature_cols]
    
    print(f"特徴量数: {len(feature_cols)}")
    print(f"訓練データサイズ: {X_train.shape}")
    print(f"テストデータサイズ: {X_test.shape}")
    print(f"特徴量: {feature_cols}")
    
    return X_train, y_train, X_test, feature_cols, label_encoders

# 前処理実行
X_train, y_train, X_test, feature_cols, label_encoders = preprocess_data(train, test)


In [ ]:
## 5. ベースラインモデルの訓練と交差検証


In [ ]:
# ベースラインモデルの定義
def train_baseline_model(X_train, y_train, n_splits=5, random_state=42):
    # ランダムフォレスト分類器
    model = RandomForestClassifier(
        n_estimators=100,
        random_state=random_state,
        n_jobs=-1
    )
    
    # Stratified K-Fold交差検証
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    cv_scores = []
    oof_predictions = np.zeros(len(X_train))
    
    print("=== 交差検証実行 ===")
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        # 訓練・検証データの分割
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # モデル訓練
        model.fit(X_fold_train, y_fold_train)
        
        # 予測
        val_predictions = model.predict_proba(X_fold_val)[:, 1]
        oof_predictions[val_idx] = val_predictions
        
        # AUCスコア計算
        auc_score = roc_auc_score(y_fold_val, val_predictions)
        cv_scores.append(auc_score)
        
        print(f"Fold {fold + 1}: AUC = {auc_score:.5f}")
    
    # 最終的なOOF AUCスコア
    oof_auc = roc_auc_score(y_train, oof_predictions)
    
    print(f"\n=== 交差検証結果 ===")
    print(f"平均AUC: {np.mean(cv_scores):.5f} (+/- {np.std(cv_scores):.5f})")
    print(f"OOF AUC: {oof_auc:.5f}")
    
    return model, oof_predictions, cv_scores, oof_auc

# ベースラインモデルの訓練
baseline_model, oof_pred, cv_scores, oof_auc = train_baseline_model(X_train, y_train)


In [ ]:
## 6. 特徴量重要度の分析


In [ ]:
# 特徴量重要度の分析
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': baseline_model.feature_importances_
}).sort_values('importance', ascending=False)

print("=== 特徴量重要度（上位10） ===")
print(feature_importance.head(10))

# 特徴量重要度の可視化
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(15)
sns.barplot(data=top_features, x='importance', y='feature')
plt.title('特徴量重要度（上位15）')
plt.xlabel('重要度')
plt.tight_layout()
plt.show()


In [ ]:
# 全データでモデルを再訓練
print("=== 全データでのモデル再訓練 ===")
final_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
final_model.fit(X_train, y_train)

# テストデータの予測
test_predictions = final_model.predict_proba(X_test)[:, 1]

print(f"テスト予測の統計:")
print(f"  平均: {test_predictions.mean():.5f}")
print(f"  最小: {test_predictions.min():.5f}")
print(f"  最大: {test_predictions.max():.5f}")
print(f"  標準偏差: {test_predictions.std():.5f}")

# 提出ファイルの作成
submission = sample_submission.copy()
submission['Drafted'] = test_predictions

# 提出ファイルの保存
submission_filename = 'baseline_submission.csv'
submission.to_csv(submission_filename, index=False)

print(f"\n=== 提出ファイル作成完了 ===")
print(f"ファイル名: {submission_filename}")
print(f"形式確認:")
print(submission.head())
print(f"サイズ: {submission.shape}")

# ベースラインスコアとの比較
print(f"\n=== ベースライン比較 ===")
print(f"目標ベースラインスコア: 0.80792")
print(f"交差検証OOF AUC: {oof_auc:.5f}")
if oof_auc > 0.80792:
    print("✅ ベースラインスコアを上回りました！")
else:
    print("❌ ベースラインスコアに届きませんでした。改善が必要です。")


In [ ]:
## 8. 結果の分析と改善ポイント


In [ ]:
# 結果の分析
print("=== ベースラインモデル総括 ===")
print(f"1. 交差検証AUC: {oof_auc:.5f}")
print(f"2. 標準偏差: {np.std(cv_scores):.5f}")
print(f"3. 使用特徴量数: {len(feature_cols)}")
print(f"4. 最重要特徴量: {feature_importance.iloc[0]['feature']}")

print(f"\n=== 改善ポイントの提案 ===")
print("1. 特徴量エンジニアリング:")
print("   - BMI、パワーウェイトレシオ等の組み合わせ特徴量")
print("   - ポジション別の正規化")
print("   - 年度トレンドの考慮")

print("\n2. 欠損値処理の改善:")
print("   - ポジション別の中央値/平均値補完")
print("   - 多重代入法の検討")
print("   - 欠損パターンの特徴量化")

print("\n3. モデルの改善:")
print("   - ハイパーパラメータチューニング")
print("   - 他のアルゴリズム（LightGBM、XGBoost）の試行")
print("   - アンサンブル手法の適用")

print("\n4. バリデーション戦略:")
print("   - 年度別の分割検討")
print("   - Holdout検証の追加")

# 予測分布の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(oof_pred, bins=30, alpha=0.7, label='OOF予測', edgecolor='black')
plt.hist(y_train, bins=30, alpha=0.7, label='実際のDrafted', edgecolor='black')
plt.xlabel('Drafted値')
plt.ylabel('頻度')
plt.title('OOF予測 vs 実際の分布')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(test_predictions, bins=30, alpha=0.7, color='green', edgecolor='black')
plt.xlabel('予測確率')
plt.ylabel('頻度')
plt.title('テスト予測の分布')

plt.tight_layout()
plt.show()

print(f"\n=== 次のステップ ===")
print("1. EDAで発見した特徴量エンジニアリングアイデアの実装")
print("2. 複数モデルでの実験")
print("3. ハイパーパラメータ最適化")
print("4. アンサンブル手法の適用")
